In [37]:
from peewee import *
from playhouse.postgres_ext import *
import datetime

db = PostgresqlDatabase('test', user='postgres', password='password', host='localhost', port=5432)


In [38]:
import logging
logger = logging.getLogger('peewee')
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)

In [39]:
class BaseModel(Model):
    class Meta:
        database = db

class User(BaseModel):
    username = TextField()

class Tweet(BaseModel):
    content = TextField()
    code = ArrayField(CharField, null=True)
    cat = CharField(null=True)
    timestamp = DateTimeField(default=datetime.datetime.now)
    user = ForeignKeyField(User, backref='tweets')

class Favorite(BaseModel):
    user = ForeignKeyField(User, backref='favorites')
    tweet = ForeignKeyField(Tweet, backref='favorites')

In [72]:
db.drop_tables([User, Tweet, Favorite])
db.execute_sql('drop AGGREGATE if EXISTS array_concat_agg(anycompatiblearray);')


('DROP TABLE IF EXISTS "favorite"', [])
('DROP TABLE IF EXISTS "favorite"', [])
('DROP TABLE IF EXISTS "tweet"', [])
('DROP TABLE IF EXISTS "tweet"', [])
('DROP TABLE IF EXISTS "user"', [])
('DROP TABLE IF EXISTS "user"', [])
('drop AGGREGATE if EXISTS array_concat_agg(anycompatiblearray);', None)
('drop AGGREGATE if EXISTS array_concat_agg(anycompatiblearray);', None)


<cursor object at 0x000001D491BC7840; closed: 0>

In [73]:
def populate_test_data():
    db.create_tables([User, Tweet, Favorite])
    db.execute_sql('CREATE or replace AGGREGATE array_concat_agg(anycompatiblearray) (   SFUNC = array_cat,   STYPE = anycompatiblearray );')
    data = (
        ('huey', ('meow', 'hiss', 'purr'), ('1001','1002'), 'cat1'),
        ('mickey', ('woof', 'whine'), ('1003'), 'cat2'),
        ('zaizee', ('hello', 'greet'), ('1005','1006', '1007'), 'cat1')
    )
    for username, tweets, code, cat in data:
        user = User.create(username=username)
        for tweet in tweets:
            print(f"Tweet = {tweet}, code = {code}")
            Tweet.create(user=user, content=tweet, code=code, cat=cat)

    # Populate a few favorites for our users, such that:
    favorite_data = (
        ('huey', ['whine']),
        ('mickey', ['purr']),
        ('zaizee', ['meow', 'purr']))
    for username, favorites in favorite_data:
        user = User.get(User.username == username)
        for content in favorites:
            tweet = Tweet.get(Tweet.content == content)
            Favorite.create(user=user, tweet=tweet)

In [74]:
populate_test_data()

('CREATE TABLE IF NOT EXISTS "user" ("id" SERIAL NOT NULL PRIMARY KEY, "username" TEXT NOT NULL)', [])
('CREATE TABLE IF NOT EXISTS "user" ("id" SERIAL NOT NULL PRIMARY KEY, "username" TEXT NOT NULL)', [])
('CREATE TABLE IF NOT EXISTS "tweet" ("id" SERIAL NOT NULL PRIMARY KEY, "content" TEXT NOT NULL, "code" VARCHAR(255)[], "cat" VARCHAR(255), "timestamp" TIMESTAMP NOT NULL, "user_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"))', [])
('CREATE TABLE IF NOT EXISTS "tweet" ("id" SERIAL NOT NULL PRIMARY KEY, "content" TEXT NOT NULL, "code" VARCHAR(255)[], "cat" VARCHAR(255), "timestamp" TIMESTAMP NOT NULL, "user_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"))', [])
('CREATE INDEX IF NOT EXISTS "tweet_code" ON "tweet" USING GIN ("code")', [])
('CREATE INDEX IF NOT EXISTS "tweet_code" ON "tweet" USING GIN ("code")', [])
('CREATE INDEX IF NOT EXISTS "tweet_user_id" ON "tweet" ("user_id")', [])
('CREATE INDEX IF NOT EXISTS "tweet_user_id" ON "twe

Tweet = meow, code = ('1001', '1002')
Tweet = hiss, code = ('1001', '1002')
Tweet = purr, code = ('1001', '1002')
Tweet = woof, code = 1003
Tweet = whine, code = 1003
Tweet = hello, code = ('1005', '1006', '1007')
Tweet = greet, code = ('1005', '1006', '1007')


In [82]:
[t for t in Tweet.select().dicts()]

('SELECT "t1"."id", "t1"."content", "t1"."code", "t1"."cat", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1"', [])
('SELECT "t1"."id", "t1"."content", "t1"."code", "t1"."cat", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1"', [])


[{'id': 1,
  'content': 'meow',
  'code': ['1001', '1002'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 0, 7, 39, 746334),
  'user': 1},
 {'id': 2,
  'content': 'hiss',
  'code': ['1001', '1002'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 0, 7, 39, 751369),
  'user': 1},
 {'id': 3,
  'content': 'purr',
  'code': ['1001', '1002'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 0, 7, 39, 753382),
  'user': 1},
 {'id': 4,
  'content': 'woof',
  'code': ['1', '0', '0', '3'],
  'cat': 'cat2',
  'timestamp': datetime.datetime(2024, 7, 6, 0, 7, 39, 759383),
  'user': 2},
 {'id': 5,
  'content': 'whine',
  'code': ['1', '0', '0', '3'],
  'cat': 'cat2',
  'timestamp': datetime.datetime(2024, 7, 6, 0, 7, 39, 764383),
  'user': 2},
 {'id': 6,
  'content': 'hello',
  'code': ['1005', '1006', '1007'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 0, 7, 39, 769382),
  'user': 3},
 {'id': 7,
  'content': 'greet',
  'code': ['1005', 

In [81]:
[t for t in Tweet.select(SQL('array_concat_agg(distinct code) as code')).group_by(Tweet.cat).dicts()]

('SELECT array_concat_agg(distinct code) as code FROM "tweet" AS "t1" GROUP BY "t1"."cat"', [])
('SELECT array_concat_agg(distinct code) as code FROM "tweet" AS "t1" GROUP BY "t1"."cat"', [])


[{'code': ['1001', '1002', '1005', '1006', '1007']},
 {'code': ['1', '0', '0', '3']}]